In [ ]:
!pip install --upgrade google-cloud-aiplatform
!gcloud auth application-default login

In [20]:
import ast

In [21]:
import base64
import vertexai
from vertexai.generative_models import GenerativeModel, SafetySetting, Part


def multiturn_generate_content():
    vertexai.init(project="407554160422", location="us-central1")
    model = GenerativeModel(
        "projects/407554160422/locations/us-central1/endpoints/5879292083369410560",
    )
    chat = model.start_chat()

    with open(INPUT_IMAGE_PATH, "rb") as image_file:
        encoded_image = base64.b64encode(image_file.read()).decode('utf-8')

    image1_1 = Part.from_data(
        mime_type="image/jpeg",
        data=base64.b64decode(encoded_image)
    )

    message = chat.send_message(
        [image1_1, """1. Determine whether this photo is a deepfake if it is deepfake return True , not False.
                      2. Tell me the reasons for your judgment

                      List a reason for judgments in JSON format.
                      only answer your determine not reasons
                      Don't use markdown syntax
                      Use this JSON schema:

                      Recipe = {'determine': bool}
                      Return: list[Recipe]
                      """],
        generation_config=generation_config,
        safety_settings=safety_settings
    )

    return message


generation_config = {
    "max_output_tokens": 8192,
    "temperature": 0,
    "top_p": 0.95,
}

safety_settings = [
    SafetySetting(
        category=SafetySetting.HarmCategory.HARM_CATEGORY_HATE_SPEECH,
        threshold=SafetySetting.HarmBlockThreshold.OFF
    ),
    SafetySetting(
        category=SafetySetting.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT,
        threshold=SafetySetting.HarmBlockThreshold.OFF
    ),
    SafetySetting(
        category=SafetySetting.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT,
        threshold=SafetySetting.HarmBlockThreshold.OFF
    ),
    SafetySetting(
        category=SafetySetting.HarmCategory.HARM_CATEGORY_HARASSMENT,
        threshold=SafetySetting.HarmBlockThreshold.OFF
    ),
]

INPUT_IMAGE_PATH = "/content/72201_1982-06-21_2015.jpg"

result = multiturn_generate_content()
print(type(result.candidates[0]))
text = result.candidates[0].content.parts[0].text.strip()

# Remove code block markers (e.g., ```json and ```)
cleaned_text = text.strip('```json').strip('```').strip()

# Convert the string representation of the list to a Python object
parsed_result = ast.literal_eval(cleaned_text)

# Extract 'determine' value
print(parsed_result[0]["determine"])

<class 'vertexai.generative_models._generative_models.Candidate'>
True


In [22]:
import time

In [23]:
import os
import ast

# real 디렉토리 경로
REAL_DIR = "/content/sample_data/test/fake"

# 디렉토리 내 파일 리스트 가져오기
file_list = sorted(os.listdir(REAL_DIR))  # 정렬된 파일 목록

# 이미지 파일만 필터링
image_extensions = {".jpg", ".jpeg", ".png", ".bmp", ".gif"}  # 이미지 확장자
image_files = [f for f in file_list if os.path.splitext(f)[1].lower() in image_extensions]

true_fake_list = [] # 딥페이크를 딥페이크로 판단한 이미지 리스트
false_fake_list = [] # 딥페이크를 원본으로 판단한 이미지 리스트
print("**** fake_directory prediction start! ****")
# 파일별 반복 실행
for file_name in image_files:
    INPUT_IMAGE_PATH = os.path.join(REAL_DIR, file_name)
    print(f"Processing file: {INPUT_IMAGE_PATH}")

    # 주어진 코드
    result = multiturn_generate_content()
    print(type(result.candidates[0]))
    text = result.candidates[0].content.parts[0].text.strip()

    # Remove code block markers (e.g., ```json and ```)
    cleaned_text = text.strip('```json').strip('```').strip()

    # Convert the string representation of the list to a Python object
    parsed_result = ast.literal_eval(cleaned_text)

    # Extract 'determine' value
    determine_value = parsed_result[0]["determine"]
    print(f"Determine value for {file_name}: {determine_value}")
    if determine_value == True:
        true_fake_list.append(file_name)
    else:
        false_fake_list.append(file_name)
    time.sleep(10)

**** fake_directory prediction start! ****
Processing file: /content/sample_data/test/fake/1086708_1952-11-04_1995.jpg
<class 'vertexai.generative_models._generative_models.Candidate'>
Determine value for 1086708_1952-11-04_1995.jpg: False
Processing file: /content/sample_data/test/fake/11310408_1923-04-17_1947.jpg
<class 'vertexai.generative_models._generative_models.Candidate'>
Determine value for 11310408_1923-04-17_1947.jpg: True
Processing file: /content/sample_data/test/fake/11505508_1966-10-06_2012.jpg
<class 'vertexai.generative_models._generative_models.Candidate'>
Determine value for 11505508_1966-10-06_2012.jpg: True
Processing file: /content/sample_data/test/fake/1170708_1943-04-29_2012.jpg
<class 'vertexai.generative_models._generative_models.Candidate'>
Determine value for 1170708_1943-04-29_2012.jpg: True
Processing file: /content/sample_data/test/fake/12370208_1940-05-06_1965.jpg
<class 'vertexai.generative_models._generative_models.Candidate'>
Determine value for 12370

TypeError: unsupported operand type(s) for +: 'int' and 'str'

In [32]:
print("true_fake :",len(true_fake_list), "개")
print(true_fake_list)

print()

print("false_fake_list", len(false_fake_list), "개")
print(false_fake_list)

true_fake : 48 개
['11310408_1923-04-17_1947.jpg', '11505508_1966-10-06_2012.jpg', '1170708_1943-04-29_2012.jpg', '12370208_1940-05-06_1965.jpg', '1261508_1974-04-26_2006.jpg', '1346208_1935-07-18_1956.jpg', '1352308_1927-04-22_2000.jpg', '14084208_1987-01-20_2009.jpg', '1482008_1968-06-08_2007.jpg', '1490108_1924-01-25_1950.jpg', '149808_1908-12-28_1940.jpg', '151408_1936-07-09_2007.jpg', '1523508_1947-12-28_1969.jpg', '16934208_1965-04-10_2006.jpg', '17066908_1998-07-09_2010.jpg', '1968808_1989-01-13_2005.jpg', '1969008_1931-09-20_2011.jpg', '2061308_1961-08-13_2007.jpg', '23702108_1963-08-27_2011.jpg', '2439308_1968-02-11_2012.jpg', '248708_1913-07-18_1960.jpg', '296008_1972-09-28_2007.jpg', '297308_1888-04-30_1941.jpg', '297408_1919-03-17_1948.jpg', '2998708_1955-01-15_1982.jpg', '318908_1956-01-02_2010.jpg', '328908_1956-09-25_2014.jpg', '36388508_1995-01-31_2011.jpg', '37337008_1994-04-21_2011.jpg', '38891708_1957-03-19_2009.jpg', '3967808_1983-02-04_2014.jpg', '40782608_1996-08-2

In [30]:
import os
import ast

# real 디렉토리 경로
REAL_DIR = "/content/sample_data/test/real"

# 디렉토리 내 파일 리스트 가져오기
file_list = sorted(os.listdir(REAL_DIR))  # 정렬된 파일 목록

# 이미지 파일만 필터링
image_extensions = {".jpg", ".jpeg", ".png", ".bmp", ".gif"}  # 이미지 확장자
image_files = [f for f in file_list if os.path.splitext(f)[1].lower() in image_extensions]

true_real_list = [] # 원본을 원본으로 판단한 이미지 리스트
false_real_list = [] # 원본을 딥페이크로 판단한 이미지 리스트
print("**** real_directory prediction start! ****")
# 파일별 반복 실행
for file_name in image_files:
    INPUT_IMAGE_PATH = os.path.join(REAL_DIR, file_name)
    print(f"Processing file: {INPUT_IMAGE_PATH}")

    # 주어진 코드
    result = multiturn_generate_content()
    print(type(result.candidates[0]))
    text = result.candidates[0].content.parts[0].text.strip()

    # Remove code block markers (e.g., ```json and ```)
    cleaned_text = text.strip('```json').strip('```').strip()

    # Convert the string representation of the list to a Python object
    parsed_result = ast.literal_eval(cleaned_text)

    # Extract 'determine' value
    determine_value = parsed_result[0]["determine"]
    print(f"Determine value for {file_name}: {determine_value}")
    if determine_value == True:
        false_real_list.append(file_name)
    else:
        true_real_list.append(file_name)
    time.sleep(10)

**** fake_directory prediction start! ****
Processing file: /content/sample_data/test/real/102100_1970-10-09_2008.jpg
<class 'vertexai.generative_models._generative_models.Candidate'>
Determine value for 102100_1970-10-09_2008.jpg: False
Processing file: /content/sample_data/test/real/1138200_1963-02-24_2014.jpg
<class 'vertexai.generative_models._generative_models.Candidate'>
Determine value for 1138200_1963-02-24_2014.jpg: True
Processing file: /content/sample_data/test/real/1142200_1980-03-17_2010.jpg
<class 'vertexai.generative_models._generative_models.Candidate'>
Determine value for 1142200_1980-03-17_2010.jpg: True
Processing file: /content/sample_data/test/real/1157400_1938-10-01_1961.jpg
<class 'vertexai.generative_models._generative_models.Candidate'>
Determine value for 1157400_1938-10-01_1961.jpg: False
Processing file: /content/sample_data/test/real/1167000_1981-08-18_2012.jpg
<class 'vertexai.generative_models._generative_models.Candidate'>
Determine value for 1167000_198

In [38]:
print("true_real_list :", len(true_real_list), "개" )
print("true_real_list", true_real_list)

print()

print("false_real_list", len(false_real_list), "개")
print(false_real_list)
print()

true_real_list : 36 개
true_real_list ['102100_1970-10-09_2008.jpg', '1157400_1938-10-01_1961.jpg', '1167000_1981-08-18_2012.jpg', '1235700_1943-03-19_1983.jpg', '153000_1940-11-29_2006.jpg', '1653100_1980-01-25_2008.jpg', '1698400_1958-12-10_2008.jpg', '1716100_1958-02-13_2013.jpg', '1741200_1967-06-28_2012.jpg', '1759400_1938-07-27_2010.jpg', '1836500_1984-01-31_2015.jpg', '1859900_1911-03-21_1969.jpg', '1878100_1958-09-13_1993.jpg', '1896500_1978-05-08_2009.jpg', '1904800_1970-02-12_2008.jpg', '305500_1940-09-18_1963.jpg', '311200_1943-09-21_2013.jpg', '311900_1959-03-29_2010.jpg', '346200_1968-12-09_2005.jpg', '37500_1944-01-23_2010.jpg', '462500_1986-02-21_2013.jpg', '489500_1970-05-05_2006.jpg', '493900_1952-08-18_2006.jpg', '575600_1979-07-13_2013.jpg', '644300_1973-02-04_2006.jpg', '67000_1947-05-13_2007.jpg', '681100_1959-10-10_2010.jpg', '69300_1950-05-11_2009.jpg', '784200_1962-06-27_2006.jpg', '787300_1967-03-16_2014.jpg', '868000_1977-04-25_2009.jpg', '882800_1967-12-07_201